In [2]:
from datautil.common import path_to_save_base
from datautil.gamelog import result_parse

In [23]:
import os
save = 'test_1000'

dn = path_to_save_base(save) + "/news/txt/leagues/"

files = os.listdir(dn)

import random
k = int(random.random() * len(files))

fn = files[k]
print(fn)

logs = open(dn + fn).readlines()

log_276.txt


In [28]:
import re
cursor = 0
inning = (0, 0)
pitcher_id = None 
batter_id = None

watch = (0,0)

stack = []
last_result = None
while cursor < len(logs):
    ln = logs[cursor]

    res_inn = re.search("(Top|Bottom) of the ([0-9]+)(st|nd|rd|th) -", ln)
    if res_inn != None:
        inning = (int(res_inn.group(2)), 0 if res_inn.group(1)=='Top' else 1)
        if inning >= watch:
            print ("-"*32)
            print ("Inning change :", inning)
        cursor += 1
        continue

    if inning < watch:
        cursor +=1
        continue
    
    res_p = re.search("Pitching: .HP <a href=\"../players/player_(\d+).html", ln)
    if res_p != None:
        pitcher_id = int(res_p.group(1))
        print ("Pitcher", pitcher_id, "for home team" if inning[1]=="Top" else "for road team")
        cursor += 1
        continue
    
    # Pinch Hitting: LHB <a href="../players/player_543.html">Morgan McCants</a>
    res_b = re.search("(Batting|Pinch Hitting): .HB <a href=\"../players/player_(\d+).html", ln)
    if res_b != None:
        if len(stack) > 0 and last_result == None:
            print("".join(["*** " + a for a in stack]))

        batter_id = int(res_b.group(2))
        print ("- Batter", batter_id, "PH" if res_b.group(1) == "Pinch Hitting" else "")
        cursor += 1
        stack = []
        last_result = None
        continue

    # groud out
    res = re.search("Ground out ([^ ]*) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- GO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # fly out --> could be sacrifice fo
    res = re.search("Fly out, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- FO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)
        if "Runner from 3rd tags up, SCORES" in logs[cursor + 1]:
            print ("--- Rec SF")
    
    # GIDP 
    res = re.search("Grounds into double play, ([^ ]*) *\((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- GO > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)
        print("----- GIDP")

    # FC
    res = re.search("Fielders Choice at ..., (.*) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- FC > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # hit single/double/triple
    res = re.search("<b>(SINGLE|DOUBLE|TRIPLE)</b>  \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- HIT({}) > type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    # hit HR
    res = re.search("<b>([A-Z]*) HOME RUN</b>  \((.*), (.*), EV (.*) MPH\), Distance : (.*) ft", ln)
    if res != None:
        last_result = "--- HR > score:{} type:{} loc:{} spd:{} dist:{}".format(*res.groups())
        print(last_result)

    # strike out
    res = re.search("Strikes out (looking|swinging)", ln)
    if res != None:
        last_result = "--- SO > {}".format(*res.groups())
        print(last_result)

    # reached on error
    res = re.search("Reached on error, E(.) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- ERR > def:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    res = re.search("Reached via error on a dropped throw from (.*), E(.) \((.*), (.*), EV (.*) MPH\)", ln)
    if res != None:
        last_result = "--- ERR (drop throw) > def:{} err:{} type:{} loc:{} spd:{}".format(*res.groups())
        print(last_result)

    if "Hit by Pitch" in ln:
        last_result = "--- HBP"
        print(last_result)

    if "Base on Balls" in ln:
        last_result = "--- BB"
        print(last_result)
        
    if "Intentional Walk" in ln:
        last_result = "--- IBB"
        print(last_result)
        
    # Bunts
    res = re.search("Bunt - Flyout to ([^ ]*)", ln)
    if res != None:
        last_result = "--- Bunt Fail > Flyout loc:{}".format(*res.groups())
        print (last_result)
        if "DP" in ln:
            print ("----- DP")
            
    if "Bunted foul, Strikeout!" in ln:
        last_result = "--- Bunt Fail > Strike out"
        print (last_result)
            
    res = re.search("Sac Bunt to ([^ ]*)", ln)
    if res != None:
        last_result = "--- Sac Bunt > loc: {}".format(*res.groups())
        print (last_result)
            
    # non PA result
    # pick off
    res = re.search("Pickoff Throw to (First|Second|Third) - Out!", ln)
    if res != None:
        last_result = "--- PICKOFF OUT > {}".format(*res.groups())
        print (last_result)
        
    # 0-1: Bunt - Flyout to 13 - DP at third! 1-3
    # 0-1: Sac Bunt to 13 - play at first, batter OUT! 1-3
    # Intentional Walk
    
    stack.append(ln)

    cursor += 1


--------------------------------
Inning change : (1, 0)
Pitcher 192 for road team
- Batter 126 
--- FO > def:F9 type:Flyball loc:9S spd:91.3
- Batter 114 
--- GO > def:4-3 type:Groundball loc:4 spd:79.4
- Batter 110 
--- GO > def:5-3 type:Groundball loc:56 spd:85.0
--------------------------------
Inning change : (1, 1)
Pitcher 100 for road team
- Batter 1297 
--- SO > swinging
- Batter 209 
--- GO > def:6-3 type:Groundball loc:6S spd:66.3
- Batter 214 
--- FO > def:F8 type:Flyball loc:8LXD spd:105.6
--------------------------------
Inning change : (2, 0)
- Batter 115 
--- SO > looking
- Batter 113 
--- SO > swinging
- Batter 1167 
--- HIT(SINGLE) > type:Flyball loc:8RS spd:98.8
- Batter 1168 
--- SO > swinging
--------------------------------
Inning change : (2, 1)
- Batter 211 
--- SO > swinging
- Batter 207 
--- GO > def:U3 type:Groundball loc:3L spd:83.8
- Batter 208 
--- SO > looking
--------------------------------
Inning change : (3, 0)
- Batter 1156 
--- FO > def:F9 type:Flybal